# Lab 12-1 many to one
### word sentiment classification 
* many to one 
* variable input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pprint import pprint

print(tf.__version__)
tf.enable_eager_execution()

1.12.0


### Prepairing dataset

In [2]:
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(idx2char)
print(char2idx)

{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [3]:
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word : len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [4]:
# padding
max_sequence = 10
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,
                       padding = 'post', truncating = 'post')

# 데이터 형태 확인
print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


In [5]:
data = tf.data.Dataset.from_tensor_slices((x_data, y_data))
data = data.shuffle(buffer_size = 4)
data = data.batch(batch_size = 2)
print(data)

<BatchDataset shapes: ((?, 10), (?,)), types: (tf.int32, tf.int32)>


### Define ManytoOne class

In [6]:
class ManytoOne(keras.Model):
    def __init__(self, num_classes, hidden_dim, max_length, dic):
        super(ManytoOne, self).__init__()
        # one-hot을 사용하기때문에
        input_dim = len(dic)
        output_dim = len(dic)
        one_hot = np.eye(len(dic))
        self.look_up = keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                                              trainable=False, mask_zero=True, input_length=max_length,
                                              embeddings_initializer=keras.initializers.Constant(one_hot))
        self.lstm_cell = keras.layers.LSTM(units=hidden_dim, return_sequences=True,
                                           return_state=True)
        self.dense = keras.layers.Dense(units=num_classes)
        
    def call(self, inputs):
        token_representation = self.look_up(inputs)        
        hidden_states, cell_states, final_hidden_state = self.lstm_cell(token_representation)
        score = self.dense(final_hidden_state)
        return score

### Create a model of ManytoOne

In [7]:
model = ManytoOne(num_classes=2, hidden_dim=16, max_length=max_sequence, dic=char2idx)

### Training

In [8]:
def loss_fn(model, x, y):
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=model(x))

lr = .1
epochs =10
opt = tf.train.AdamOptimizer(learning_rate = lr)

In [9]:
tr_loss_hist = []

for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0
    
    for x_mb, y_mb in data:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, x=x_mb, y=y_mb)
        grads = tape.gradient(target=tr_loss, sources=model.variables)
        opt.apply_gradients(grads_and_vars=zip(grads, model.variables))
        avg_tr_loss += tr_loss
        tr_step += 1
    else:
        avg_tr_loss /= tr_step
        tr_loss_hist.append(avg_tr_loss)
    
    print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, tr_loss : 2.562
epoch :   2, tr_loss : 0.652
epoch :   3, tr_loss : 0.629
epoch :   4, tr_loss : 0.412
epoch :   5, tr_loss : 0.180
epoch :   6, tr_loss : 0.023
epoch :   7, tr_loss : 0.000
epoch :   8, tr_loss : 0.000
epoch :   9, tr_loss : 0.000
epoch :  10, tr_loss : 3.811


### Checking performance

In [10]:
yhat = model(tf.convert_to_tensor(x_data))
yhat = np.argmax(yhat, axis=-1)
print('accuracy : {:.2%}'.format(np.mean(yhat == y_data)))

accuracy : 100.00%
